# Problem Simulation Tutorial

In [1]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.6.0'

Before configuring and solving a problem with real data, papers such as :ref:`references:Armstrong (2016)` recommend performing Monte Carlo analysis on simulated data to verify that it is possible to accurately estimate model parameters. For example, before configuring and solving the example problems in the prior tutorials, it may have been a good idea to simulate data according to the assumed models of supply and demand. During such Monte Carlo anaysis, the data would only be used to determine sample sizes and perhaps to choose reasonable true parameters.

Simulations are configured with the :class:`Simulation` class, which requires many of the same inputs as :class:`Problem`. The two main differences are:

1. Variables in formulations that cannot be loaded from `product_data` or `agent_data` will be drawn from independent uniform distributions.
2. True parameters and the distribution of unobserved product characteristics are specified.

First, we'll use :func:`build_id_data` to build market and firm IDs for a model in which there are $T = 50$ markets, and in each market $t$, a total of $J_t = 20$ products produced by $F_t = 10$ firms.

In [2]:
id_data = pyblp.build_id_data(T=50, J=20, F=10)

Next, we'll create an :class:`Integration` configuration to build agent data according to a Gauss-Hermite product rule that exactly integrates polynomials of degree $2 \times 9 - 1 = 17$ or less.

In [3]:
integration = pyblp.Integration('product', 9)
integration

Configured to construct nodes and weights according to the level-9 Gauss-Hermite product rule.

We'll then pass these data to :class:`Simulation`. We'll use :class:`Formulation` configurations to create an $X_1$ that consists of a constant, prices, and an exogenous characteristic; an $X_2$ that consists only of the same exogenous characteristic; and an $X_3$ that consists of the common exogenous characteristic and a cost-shifter.

In [4]:
simulation = pyblp.Simulation(
   product_formulations=(
       pyblp.Formulation('1 + prices + x'),
       pyblp.Formulation('0 + x'),
       pyblp.Formulation('0 + x + z')
   ),
   beta=[1, -2, 2],
   sigma=1,
   gamma=[1, 4],
   product_data=id_data,
   integration=integration,
   seed=0
)
simulation

Dimensions:
 N     T    K1    K2    K3    MD    MS 
----  ---  ----  ----  ----  ----  ----
1000  50    3     1     2     5     6  

Formulations:
       Column Indices:           0      1       2  
-----------------------------  -----  ------  -----
 X1: Linear Characteristics      1    prices    x  
X2: Nonlinear Characteristics    x                 
  X3: Cost Characteristics       x      z          

Nonlinear Coefficient True Values:
Sigma:      x     
------  ----------
  x      +1.0E+00 

Beta True Values:
    1         prices        x     
----------  ----------  ----------
 +1.0E+00    -2.0E+00    +2.0E+00 

Gamma True Values:
    x           z     
----------  ----------
 +1.0E+00    +4.0E+00 

When :class:`Simulation` is initialized, it constructs :attr:`Simulation.agent_data` and simulates all :attr:`Simulation.product_data` except for prices and shares, which are initialized as zeros and need to be solved for.

The excluded instruments in :attr:`Simulation.product_data` include basic instruments computed with :func:`build_blp_instruments` that are functions of all exogenous numerical variables in the problem. In this example, excluded demand-side instruments are the cost-shifter `z` and traditional BLP instruments constructed from `x`. Excluded supply-side instruments are traditional BLP instruments constructed from `x` and `z`.

The :class:`Simulation` can be further configured with other arguments that determine how unobserved product characteristics are simulated and how marginal costs are specified.

Since at this stage prices and shares are all zeros, we still need to solve the simulation with :meth:`Simulation.solve`. This method computes synthetic prices and shares. Just like :meth:`ProblemResults.compute_prices`, to do so it iterates over the $\zeta$-markup equation from :ref:`references:Morrow and Skerlos (2011)`.

In [5]:
simulation_results = simulation.solve()
simulation_results

Simulation Results Summary:
Computation  Fixed Point  Contraction
   Time      Iterations   Evaluations
-----------  -----------  -----------
  0:00:00        706          706    

Now, we can try to recover the true parameters by creating and solving a :class:`Problem`. By default, the convenience method :meth:`SimulationResults.to_problem` uses the same formulations and unobserved agent data as the simulation, so estimation is relatively easy. However, we'll choose starting values that are half the true parameters so that the optimization routine has to do some work. Note that since we're jointly estimating the supply side, we need to provide an initial value for the linear coefficient on prices because this parameter cannot be concentrated out of the problem (unlike linear coefficients on exogenous characteristics).

In [6]:
problem = simulation_results.to_problem()
problem

Dimensions:
 N     T    K1    K2    K3    MD    MS 
----  ---  ----  ----  ----  ----  ----
1000  50    3     1     2     5     6  

Formulations:
       Column Indices:           0      1       2  
-----------------------------  -----  ------  -----
 X1: Linear Characteristics      1    prices    x  
X2: Nonlinear Characteristics    x                 
  X3: Cost Characteristics       x      z          

In [7]:
results = problem.solve(
   sigma=0.5 * simulation.sigma, 
   pi=0.5 * simulation.pi,
   beta=[None, 0.5 * simulation.beta[1], None]
)
results

Problem Results Summary:
                   Cumulative    Cumulative     Final    Cumulative   Cumulative   Gradient
Cumulative  GMM   Optimization  Optimization  Objective  Fixed Point  Contraction  Infinity
   Time     Step   Iterations   Evaluations     Value    Iterations   Evaluations    Norm  
----------  ----  ------------  ------------  ---------  -----------  -----------  --------
 0:00:44     2         15            32       +9.3E+03      88470       266914     +4.7E-05

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      x     
------  ----------
  x      +1.5E+00 
        (+4.5E-01)

Beta Estimates (Robust SEs in Parentheses):
    1         prices        x     
----------  ----------  ----------
 +1.1E+00    -2.0E+00    +1.8E+00 
(+1.2E-01)  (+2.3E-02)  (+1.7E-01)

Gamma Estimates (Robust SEs in Parentheses):
    x           z     
----------  ----------
 +9.3E-01    +4.2E+00 
(+8.5E-02)  (+7.4E-02)

The parameters seem to have been estimated reasonably well.

In [8]:
np.c_[simulation.beta, results.beta]

array([[ 1.        ,  1.05244586],
       [-2.        , -1.96850863],
       [ 2.        ,  1.8408699 ]])

In [9]:
np.c_[simulation.gamma, results.gamma]

array([[1.        , 0.9327637 ],
       [4.        , 4.17298673]])

In [10]:
np.c_[simulation.sigma, results.sigma]

array([[1.        , 1.48688668]])

In addition to checking that the configuration for a model based on actual data makes sense, the :class:`Simulation` class can also be a helpful tool for better understanding under what general conditions BLP models can be accurately estimated. Simulations are also used extensively in pyblp's test suite.